# Hash Tables (Google)

Implement an LRU (Least Recently Used) cache. It should be able to be initialized with a cache size `n`, and contain the following methods (each of which should run in O(1) time:
- `set(key, value)`: sets `key` to `value`. If there are already `n` items in the cache, and we are adding a new item, then it should remove the least recently used item.
- `get(key)`: gets the `value` at `key`. If no such `key` exists, return null.

In [ ]:
dct = {
    'one': [1, 2, 3],
    'two': 8
}















### Solution

To implement both these methods in constant time, we'll need to use a hash table (dictionary) along with a linked list. The hash table will map `keys` to `nodes` in the linked list, and the linked list will be ordered from most recently used to least recently used.

Then, for `set`:
- First look at our current capacity. If it's `< n`, then create a `node` with the `value`, set it as the `head`, and add it as an entry in the dictionary.
- If it's equal to `n`, then add our `node` as usual, but also evict the least frequently used `node` by deleting the `tail` of our linked list and also removing the entry from our dictionary. We'll need to keep track of the `key` in each `node` so that we know which entry to evict.

For `get`:
- If the `key` doesn't exist in our dictionary, then return null.
- Otherwise, look up the relevant `node` through the dictionary. Before returning it, update the linked list by moving the node to the `front` of the list.

To simplify our implementation, we can use the following tricks:
- Using dummy nodes for the `head` and `tail` of our list, which will simplify creating the list when nothing's initialized.
- Implementing the helper class `LinkedList` to reuse code when adding and removing `nodes` to our linked list.
- When we need to bump a `node` to the back of list (like when we fetch it), we can just remove it and add it back to the list.

Putting it all together, we have the following solution:

In [1]:
class Node:
    def __init__(self, key, val, prev=None, nxt=None):
        self.key = key
        self.val = val
        self.prev = prev
        self.next = nxt

        
class LinkedList:
    def __init__(self):
        # Dummy nodes for head and tail of list
        self._head = Node(None, None)
        self._tail = Node(None, None, self._head)
        
        # Finish connecting up the dummy nodes
        self._head.next = self._tail
    
    def get_head(self):
        return self._head.next
    
    def get_tail(self):
        return self._tail.prev
    
    def add(self, key, value):
        node = Node(key, value, self._head, self._head.next)
        
        # Update references and return node
        self._head.next = node
        node.next.prev = node
        return node
    
    def remove(self, node):
        prev = node.prev
        nxt = node.next
        prev.next = nxt
        nxt.prev = prev

        
class LRUCache:
    def __init__(self, n):
        self.n = n
        self.dict = {}
        self.list = LinkedList()
    
    def set(self, key, val):
        if key in self.dict:
            del self.dict[key]
        
        self.dict[key] = self.list.add(key, val)
        if len(self.dict) > self.n:
            tail = self.list.get_tail()
            self.list.remove(tail)
            del self.dict[tail.key]
    
    def get(self, key):
        if key in self.dict:
            node = self.dict[key]
            
            # Move node to front of list and return value
            self.list.remove(node)
            self.list.add(node.key, node.val)
            return node.val